# Part 1

In [4]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.regression.gmm import GMM
from scipy import stats

def print_gmm_results(results, n_obs, title):
    """
    Print GMM results in a simple formatted way
    """
    print("=" * 50)
    print(title)
    print("=" * 50)
    print("Dep. Variable:           y")
    print(f"No. Observations:       {n_obs}")
    print(f"Hansen J:               {results.jtest}")
    print("\nParameters:")
    print("-" * 50)
    print(f"{'Parameter':<12}{'Coef':<10}{'Std Err':<10}{'z':<10}{'P>|z|':<10}")
    print("-" * 50)
    
    std_errors = np.sqrt(np.diag(results.cov_params()))
    
    param_names = ['Constant', 'Inv Turn', 'Op Profit', 'Inter Eff']
    
    for i, (name, coef, std_err) in enumerate(zip(param_names, results.params, std_errors)):
        z_stat = coef / std_err
        p_val = 2 * (1 - stats.norm.cdf(abs(z_stat)))
        print(f"{name:<12}{coef:<10.4f}{std_err:<10.4f}{z_stat:<10.4f}{p_val:<10.4f}")
    
    print("\n")

# Load and prepare data
url = "https://raw.githubusercontent.com/neilaxu/schulich_data_science/main/Predictive%20Modelling/Assignment%202/midterm_partone.csv"
data = pd.read_csv(url)

# Prepare arrays
y_vals = np.array(data["Stock Change"])
x_vals = np.array(data[["Inventory Turnover", "Operating Profit", "Interaction Effect"]])
iv_vals = np.array(data[["Current Ratio", "Quick Ratio", "Debt Asset Ratio"]])
n_obs = len(y_vals)

# Initial values
beta0 = np.array([0.1, 0.1, 0.1, 0.1])

# Original GMM class
class original_gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        error = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        
        error0 = error 
        error1 = error * exog[:,1]
        error2 = error * exog[:,2]
        error3 = error * inst[:,0]
        error4 = error * inst[:,1]
        error5 = error * inst[:,2]

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g

# Modified GMM class with scalar delta
class modified_gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument
        
        # Scalar delta
        delta = 1.0
        
        error = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        
        error0 = error - delta
        error1 = error * exog[:,1] - delta
        error2 = error * exog[:,2] - delta
        error3 = error * inst[:,0] - delta
        error4 = error * inst[:,1] - delta
        error5 = error * inst[:,2] - delta

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g

# Estimate models
original_model = original_gmm(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=6, k_params=4)
modified_model = modified_gmm(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=6, k_params=4)

original_results = original_model.fit(beta0)
modified_results = modified_model.fit(beta0)

# Print results
print_gmm_results(original_results, n_obs, "Original GMM Results")
print_gmm_results(modified_results, n_obs, "Modified GMM Results (with δ=1)")

# Print comparison
print("Parameter Comparison:")
print("=" * 50)
comparison = pd.DataFrame({
    'Original GMM': original_results.params,
    'Modified GMM': modified_results.params,
    'Difference': modified_results.params - original_results.params
}, index=['Constant', 'Inventory Turnover', 'Operating Profit', 'Interaction Effect'])
print(comparison.round(4))


Optimization terminated successfully.
         Current function value: 0.000046
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.000373
         Iterations: 7
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.549867
         Iterations: 8
         Function evaluations: 12
         Gra

# Mathematical Derivation of Modified GMM Model

## 1. Starting Point: True Model Structure

The true model is given by:

$Y = XB + \theta Z + E$

where:
- $Y$ is the stock return (dependent variable)
- $X$ is the matrix of explanatory variables (Inventory Turnover, Operating Profit, Interaction Effect)
- $B$ is the coefficient vector
- $\theta$ is a constant coefficient
- $Z$ is the omitted variable vector
- $E$ is the error term

## 2. Original GMM Moment Conditions

In standard GMM estimation without considering bias, we have:

For explanatory variables:
$X^T(Y - XB) = 0$

For instrumental variables:
$Z^T(Y - XB) = 0$

## 3. Expert's Claim and Bias Term

The expert (David Berman) claims:

$X^T(Y - XB) = \delta$

Let's derive what this means:

1. Substitute the true model:
   $X^T(Y - XB) = X^T((XB + \theta Z + E) - XB) = \delta$

2. Simplify:
   $X^T(\theta Z + E) = \delta$

This shows that $\delta$ captures the combined effect of:
- The omitted variable ($\theta Z$)
- The correlation between X and the error term (E)

## 4. Modified Moment Conditions

To incorporate this bias, we modify our moment conditions:

1. For explanatory variables:
   $X^T(Y - XB) - \delta = 0$

2. For instruments:
   $Z^T(Y - XB) - \delta = 0$

## 5. Implementation Implications

This modification has several important implications:

1. The scalar $\delta$ appears in all moment conditions, reflecting systematic bias

2. The moment conditions for both explanatory variables and instruments are adjusted by the same $\delta$, because:
   - The bias affects all moment conditions similarly
   - The original correlation structure is preserved

3. The modified estimator will:
   - Account for the systematic bias claimed by the expert
   - Potentially provide different parameter estimates from standard GMM
   - Still allow for consistent estimation under the modified conditions

## 6. Connection to Original Code

In our implementation, this translates to:

```python
# Original moment conditions:
error0 = error
error1 = error * exog[:,1]
# etc...

# Modified moment conditions:
error0 = error - delta
error1 = error * exog[:,1] - delta
# etc...
```

where `delta = 1.0` represents the expert's claimed bias.

# Analysis of GMM Results to Evaluate Expert's Claim

## 1. Key Changes in Estimates

Comparing the original and modified GMM (with δ=1), we observe significant changes:

1. **Constant term**:
   - Changed from -0.0200 to 0.4257
   - Became highly significant (p-value < 0.0001)

2. **Inventory Turnover**:
   - Changed from 0.0011 to -0.0216
   - Changed from marginally significant (p=0.0654) to highly significant (p < 0.0001)
   - Notably changed sign from positive to negative

3. **Operating Profit**:
   - Changed from -0.1071 to -1.0195
   - Magnitude increased substantially (~9.5 times larger)
   - Remained statistically significant

4. **Interaction Effect**:
   - Changed from 0.0011 to 0.0116
   - Magnitude increased (~10.5 times)
   - Remained statistically significant

## 2. Statistical Justification

The expert's claim appears to be statistically justified for several reasons:

1. **Significant Parameter Changes**:
   - All coefficients show substantial changes when accounting for the bias
   - The changes are statistically significant (based on z-statistics)
   - The modified model shows stronger statistical significance across all parameters

2. **Economic Significance**:
   - The modified model aligns better with the expert's theory about inventory management
   - The negative coefficient on Inventory Turnover in the modified model (-0.0216) suggests that after accounting for bias, higher inventory turnover is associated with lower stock returns

3. **Model Fit**:
   - Both models converged successfully
   - The coefficients in the modified model are more strongly significant, suggesting better identification of the relationships

## 3. Conclusion

The expert's claim about bias in moment conditions appears to be statistically justified because:

1. The incorporation of the bias term δ leads to materially different estimates
2. The modified estimates are more statistically significant
3. The direction and magnitude of changes align with economic intuition about inventory management

This suggests that omitting the bias term might have led to underestimation of the true relationships in the original model.